In [15]:
import numpy as np 
import utils
import xarray as xr

In [11]:
# AR6 
ECS = 3 #K 
TCR = 1.8 #K 

# IGCC 2023, https://essd.copernicus.org/articles/16/2625/2024/
# all forcing in 2023 rel. to 1750
F_anthro = 2.79 #W/m^2 [1.78 - 3.61]  
F_CO2 = 2.28 #W/m^2 
CO2_ppm = 419 #ppm

# GCB, https://essd.copernicus.org/articles/15/5301/2023/essd-15-5301-2023.pdf
Cemit = 740 #GtC (Table 8 number for 1750-2022 + 10 GtC for 2023)

Af = 0.2 
GtC_to_ppm = 2.14
CO2_pre = 280 

F_0 = F_CO2/np.log(CO2_ppm/CO2_pre)
CO2_f = Cemit/GtC_to_ppm*Af + CO2_pre
F_f = F_0*np.log(CO2_f/CO2_pre)
ZEC_hist = F_f/F_anthro/(TCR/ECS) - 1

In [12]:
output = f"""  
-----------------------Supporting_calculations-----------------------------------------------
Final radiative forcing absent future emissions (F_f): {F_f:.1f} W/m$^2$
Historical ZEC/Tze analytical estimate: {ZEC_hist:.1f}
---------------------------------------------------------------------------------------------
"""

# with open(snakemake.output.computations_file, "a", encoding="utf-8") as file: 
#     file.write(output)                                                                  
                                                                                                      

#print("Wrote supporting calculations to:", snakemake.output.computations_file) 

In [13]:
results_dir = utils.get_path('results')

In [17]:
ds = xr.open_mfdataset(results_dir+"/fair/*net-zero-zec.nc")

In [18]:
aerosol_forcing = ds.forcing.sel(specie=['Aerosol-radiation interactions','Aerosol-cloud interactions']).sum("specie")

In [19]:
for scenario in ds.scenario.values:
    aerosol = aerosol_forcing.sel(timebound=utils.get_net_zero_GHG_time(scenario), scenario=scenario,
                                  method="bfill")

    median = aerosol.quantile(0.5, dim="config")
    lower_bound = aerosol.quantile(0.05, dim="config")
    upper_bound = aerosol.quantile(0.95, dim="config")
    
    print(f"Scenario: {scenario}")
    print(f"Median: {median.values:.2f}")
    print(f"5-95% Confidence Bounds: ({lower_bound.values:.2f}, {upper_bound.values:.2f})\n")

Scenario: ssp119
Median: -0.13
5-95% Confidence Bounds: (-0.25, 0.05)

Scenario: ssp126
Median: -0.12
5-95% Confidence Bounds: (-0.24, 0.02)

Scenario: ssp534-over
Median: -0.20
5-95% Confidence Bounds: (-0.54, -0.03)

